# Gestion de la mémoire en Rust 🦀🦀


[Pierre-Antoine Champin](https://champin.net/) (W3C/Inria/Lyon 1)

http://github.com/pchampin/rust-envol-2025

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/2.0/fr/"><img alt="Contrat Creative Commons" style="border-width:0" src="http://i.creativecommons.org/l/by-nc-sa/2.0/fr/88x31.png" /></a>

## Rappel: la pile et le tas

La pile

- contient les données des variables locales,
- dont la taille est connue à la compilation (statique).
- Elle est gérée de manière *dernier arrivé, premier parti* automatiquement, dans tous les langages

Le tas

- contient les données dont la taille n'est connue qu'à l'exécution (dynamique),
- allouées/libérées de manière moins prédictible.
- Il est géré différemment selon les langages.

### Gestion du tas

En C/C++
- les dévelopeur⋅euse⋅s ont la charge de libérer la mémoire
- ce qui est parfois (souvent ?) une source d'erreurs (mémoire libérée trop tôt, trop tard, jamais...).

En Java
- le ramasse-miettes (*garbage collector*) a  la charge de libérer la mémoire
- ce qui peut causer des problèmes de performance.

### Gestion du tas

En Rust

- le **compilateur** a la charge de libérer la mémoire.
- Les dévelopeur⋅euse⋅s doivent lui fournir suffisament d'information,
- grâce aux concepts de **propriété** et d'**emprunt**. 🦀🦀

## Sémantique de l'affectation

Dans la plupart des langages de programmation,
la sémantique de l'affectation est une *copie*.

```c
v1 = init();
v2 = v1;
// v1 et v2 contiennent "la même chose"
```

En Rust, la semantique de l'affectation est un *déplacement*

* conceptuellement, les données sont déplacées de `v1` vers `v2` ;
* après l'affectation, `v1` est considérée comme non-initialisée.

### Exemple: le type `Vec`

In [2]:
let v1: Vec<i32> = vec![22, 44, 66];

<div style="text-align: center">
<img alt="A Vec in memory" src="images/trpl04-01.svg" style="display: inline; width: 12em;">
</div>

### Avec la sémantique "copie" (C)

```c
Vec v1 = init_vec();
Vec v2 = v1;
```

<div style="text-align: center"> <img alt="Shallow copy" src="images/trpl04-02.svg" style="display: inline; width: 12em;"> </div>

### Avec la sémantique "déplacement" (Rust) 🦀

In [3]:
let v1: Vec<i32> = vec![22, 44, 66];
println!("v1={v1:?}");
let v2 = v1;
println!("v2={v2:?}");
//println!("v1={v1:?}"); // NE COMPILE PAS

v1=[22, 44, 66]
v2=[22, 44, 66]


<div style="text-align: center">
<img alt="After a move assignment" src="images/trpl04-04.svg" style="display: inline; width: 12em;">
</div>

### Copie (profonde) explicite

In [4]:
let v1: Vec<i32> = vec![22, 44, 66];
let v2 = v1.clone(); // copie (profonde) explicite
println!("v1={v1:?} v2={v2:?}");

v1=[22, 44, 66] v2=[22, 44, 66]


<div style="text-align: center">
<img alt="After a deep copy" src="images/trpl04-03.svg" style="display: inline; width: 12em;">
</div>

### Copie implicite

Bien que la sémantique "déplacement" soit la règle,
il y a des exceptions.

En particulier, l'affectation des types primitifs (entiers, flottants, booléens...)
a la sémantique "copie",
car le compilateur sait que cette copie ne pose pas de problème.

In [5]:
let x = 42;
let y = x; // copie plutôt qu'un déplacement
println!("x={x} y={y}");

x=42 y=42


Plus généralement,
tout type qui implémente [trait `Copy`](https://doc.rust-lang.org/stable/std/marker/trait.Copy.html)
utilise la sémantique "copie" pour les affectations.

Les types définis par l'utilisateur peuvent implémenter le trait `Copy`,
mais ça n'est pas toujours possible (par exemple, le type `Vec`).

## Propriété 🦀🦀

* Toute donnée est **possédée** par exactement une variable (ou un champs de structure)

* Conceptuellement, l'affectation d'une variable à une autre *déplace* les données,
  et **transfère la propriété** à la variable destination.
  
* Lorsqu'une variable n'est plus utilisée (qu'elle devient hors de portée),
  si elle possède encore des données, ces données sont **libérées**.

<div style="text-align: center">
<img alt="A Vec in memory" src="images/trpl04-01.svg" style="display: inline; width: 12em;">
</div>

In [6]:
fn fibo(nb: usize) -> Vec<i32> {
    let mut v1 = Vec::with_capacity(nb); // 1. le Vec est créé ici
    for i in 0..nb {
        if i < 2 { v1.push(1); }
        else     { v1.push(v1[i-2] + v1[i-1]); }
    }
    v1                                   // 2. sa propriété est transmise à l'appelant (main)
}

fn main() { 
    let n = 7;
    let v2 = fibo(n);                    // 3. le Vec retourné par `fibo` est déplacé dans v2
    println!("v2={v2:?}");
    let s = somme(v2);                     // 4. le Vec est déplacé vers le paramètre de la fonction `somme`
    println!("La somme des {n} premiers nombres de Fibonnaci est {s}");
    //println!("{:?}", v2);             // NE COMPILE PAS
}

fn somme(v3: Vec<i32>) -> i32 {
    v3.iter().sum()
}                                        // 5. en sortant de la fonction `sum`,
                                         //    `v3` possède encore le Vec, qui est donc libéré

main();

v2=[1, 1, 2, 3, 5, 8, 13]
La somme des 7 premiers nombres de Fibonnaci est 33


## Emprunt 🦀🦀

In [7]:
let v1: Vec<i32> = vec![22, 44, 66];
println!("A. v1={v1:?}");
{
    let v2 = &v1;
    println!("B. v2={v2:?}");
}
println!("C. v1={v1:?}");

A. v1=[22, 44, 66]
B. v2=[22, 44, 66]
C. v1=[22, 44, 66]


<div style="text-align: center">
<img alt="A reference is a non-owning pointer" src="images/trpl04-05.svg" style="display: inline; width: 16em;">
</div>

Pour toute variable `v` de type `T`, `&v` permet d'**emprunter** les données de v.

* `&v` est aussi appelé une **référence**, son type est noté `&T`.
* En interne, les références sont des pointeurs.
* Conceptuellement, une référence ne *possède pas* les données vers lesquelle elle pointe.
* Libérer une référence ne libère donc pas les données pointées.
* Plusieurs références vers les mêmes données peuvent co-exister.

In [8]:
let mut v1: Vec<i32> = vec![22, 44, 66];
println!("A. v1={v1:?}");
{
    let v2 = &v1;
    let v3 = &v1;
    println!("B. v1={v1:?} v2={v2:?} v3={v3:?}");
}
println!("C. v1={v1:?}");

A. v1=[22, 44, 66]
B. v1=[22, 44, 66] v2=[22, 44, 66] v3=[22, 44, 66]
C. v1=[22, 44, 66]


NB : les données empruntées avec une référence `&T` sont immutable.

In [9]:
let mut v1: Vec<i32> = vec![22];
v1.push(44);
println!("A. v1={v1:?}");
{
    let v2 = &v1;
    //v2.push(66); // NE COMPILE PAS
    println!("B. v2={v2:?}");
}
v1.push(88);
println!("C. v1={v1:?}");

A. v1=[22, 44]
B. v2=[22, 44]
C. v1=[22, 44, 88]


Pour que l'exemple précédent fonctionne entièrement,
il nous faut une **référence mutable**,
de type `&mut T`.

In [10]:
let mut v1: Vec<i32> = vec![22];
v1.push(44);
println!("A. v1={v1:?}");
{
    let v2 = &mut v1;
    v2.push(66);
    println!("B. v2={v2:?}");
}
v1.push(88);
println!("C. v1={v1:?}");

A. v1=[22, 44]
B. v2=[22, 44, 66]
C. v1=[22, 44, 66, 88]


### Règle d'or pour les références

On ne peut pas avoir en même temps

* plusieurs références mutables, ni
* une référence mutable et une (ou plusieurs) références immutables

vers la même valeur, pour éviter

* que deux partie du code ne modifient les mêmes données simultanément, ou
* qu'une partie du code lise des données en cours de modification.

Cette règle est vérifiée par le **borrow checker** (un composant du compilateur).

In [11]:
let mut v1: Vec<i32> = vec![22];
v1.push(44);
println!("A. v1={v1:?}");
{
    let v2 = &v1;
    //v1.push(55);  // NE COMPILE PAS
    println!("B. v2={v2:?}");
}
{
    let v3 = &mut v1;
    v3.push(66);
    //v1.push(77);  // NE COMPILE PAS
    println!("C. v3={v3:?}");
}
println!("D. v1={v1:?}");
v1.push(88);
println!("E. v1={v1:?}");

A. v1=[22, 44]
B. v2=[22, 44]
C. v3=[22, 44, 66]
D. v1=[22, 44, 66]
E. v1=[22, 44, 66, 88]


### Déréférence avec `*`

* L'opérateur `*` est l'inverse de l'opérateur `&`
* Il permet de passer du type `&T` ou `&mut T` au type `T`

In [12]:
{
    let mut a: i32 = 42;
    let mut b: &mut i32 = &mut a;
    let c: i32 = *b; // copie la valeur empruntée par b (car le type le permet)
    *b = 43;         // modifie la valeur empruntée mutablement
    println!("a={a} c={c}");
};

a=43 c=42


### Référence ≠ pointeur

* En interne, ce sont bien des pointeurs.
* Mais dans leur utilisation, c'est la *valeur pointée* qui est prise en compte.

In [13]:
let v = [42, 42];
let test = (&v[0] == &v[1]);
test

true

* Mieux vaut penser `&T` comme "une valeur de `T` empruntée à quelqu'un d'autre"
  que comme "un pointeur vers un `T`".
  
  * D'ailleurs, une référence ne peut jamais être un pointeur null. 🦀
  
* Les types `T` et `&T` sont toutefois des types différents.

## Comment choisir le type des paramètres d'une fonction

In [14]:
// ma fonction n'a besoin que de lire les données: &T
fn count_zeros(v: &Vec<i32>) -> usize {
   v.iter().filter(|i| i==&&0).count()
} // NB: on préférerra en général  v: &[i32]

In [15]:
// ma fonction n'a besoin que de lire les données,
// de taille modeste et avec la sémantique "copie": T
fn fact(n: usize) -> usize {
    if n <= 1 { 1 } else { n*fact(n-1) }
}

In [16]:
// ma fonction a besoin de modifier les données: &mut T
fn add_one(v: &mut Vec<i32>) {
    for i in v { *i += 1; }
}

In [17]:
// ma fonction "consomme" les données: T
fn add_two(mut v: Vec<i32>) -> Vec<i32> {
    v.into_iter().map(|x| x+2).collect()
}

## Longévité (*lifetime*)

Le *borrow checker* doit vérifier qu'aucun emprunt de données ne survit au propriétaire de ces données.

(puisqu'après la "mort" du propriétaire, les données sont libérées et la référence pointerait "dans le vide")


In [18]:
{
    let owner1 = 22;
    let mut borrower: Vec<&i32> = vec![&owner1, &owner1];
    println!("A. {borrower:?}");
    {
        let owner2 = 44;
        borrower.push(&owner2); // NE COMPILE PAS
        println!("B. {borrower:?}");
    }
    // ici, 'owner2' n'existe plus (hors de portée)
    println!("C. {borrower:?}");
};

Error: `owner2` does not live long enough

Dans certaines situations, le compilateur a besoin d'indications explicites sur les longévités respectives des différentes références (« qui vit aussi longtemps que qui ? »).

In [19]:
// CE CODE NE COMPILE PAS

// retoure les valeurs de t1 absentes de t2
fn minus(t1: &[i32], t2: &[i32]) -> Vec<&i32> {
    let mut v = vec![];
    for i in t1 {
        if !t2.contains(i) {
            v.push(i)
        }
    }
    v
}

Error: missing lifetime specifier

La solution à ce problème (comme suggéré par le compilateur)
consiste à expliciter les longévités respective des emprunts,
avec des *lifetime parameters*.

In [20]:
// retoure les valeurs de t1 absentes de t2
fn minus<'a, 'b>(t1: &'a [i32], t2: &'b [i32]) -> Vec<&'a i32> {
    //  ^^^^^^^^      ^^             ^^                ^^
    // indique que l'emprunt retourné
    // ne peut pas vivre plus longtemps que v1 (même longévité 'a),
    // mais n'est pas impacté par v2 (longévité différente 'b)
    let mut v = vec![];
    for i in t1 {
        if !t2.contains(i) {
            v.push(i)
        }
    }
    v
}

Dans les cas courant, il n'est pas nécessaire de spécifier les longévités;
Rust applique des heuristiques correspondant aux cas les plus courants.

En particulier, lorsqu'il n'y a qu'une seule référence en entrée, et une seule référence en sortie,
elles sont supposée avoir la même longévité.

In [21]:
// retourne uniquement les valeurs de t1 inférieures à max
fn all_lower_than(t1: &[i32], max: i32) -> Vec<&i32> {
// équivalent à
// fn borrow_items<'a>(v1: &'a [i32]) -> Vec<&'a i32> {
    t1.iter()
      .filter(|i| i<=&&max)
      .collect()
}